<a href="https://colab.research.google.com/github/sjsjlee/study_financial_data_analysis/blob/main/NCAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NCAV(Net Current Asset Value)#


1.레벨 : 초급

2.스타일 : 밸류

3.기대CAGR : 20%이상

4.종목개수 : 20~30개

5.매수전략 : 유동자산-총부채>시가총액*1.5, 세후이익>0

|레벨|스타일|기대CAGR|종목개수|매수전략
|----|------|--------|--------|-------------------------------------|
|초급|밸류|20%이상|20~30개|유동자산-총부채>시가총액*1.5, 세후이익 >0


#Basic Setting#


In [ ]:
import pandas as pd
from tqdm import tqdm #상태진행률
from bs4 import BeautifulSoup #사이트 정보 추출
import urllib.request #url_library 안에 있는 request 가져옴(url 열기 위한 확장가능 라이브러리)
#!pip install bs4(설치 안되면 작성)
#!pip install urllib

In [ ]:
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'

sample_code = '035720' #카카오

In [ ]:
fs = urllib.request.urlopen(FS_URL % sample_code).read() #urllibrary의 request(객체, 클래스)사용, urlopen 함수 사용해 FS_URL 불러옴, 마지막으로 read()함수로 url 안에 담긴 내용 불러옴
fs_soup = BeautifulSoup(fs, 'html.parser') #fs를 html.parser 사용해 변환(압축 해제)

snap = urllib.request.urlopen(SNAP_URL % sample_code).read()
snap_soup = BeautifulSoup(snap, 'html.parser')

#HTML Parser란 HTML 문법규칙을 기초로 웹페이지의 내용을 해석하고, 의미와 구조를 분석하는 프로그램입니다.

In [ ]:
print(FS_URL % sample_code)

https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A035720&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701


#Market Value(시가총액)#


*html 태그
- 클래스로 묶여있음
- id는 하나

여기서 시가총액은 div 클래스 > td 클래스 이런식으로 묶여있음

In [ ]:
mv_cells = snap_soup.find('div', {'id' : 'svdMainGrid1'}).find_all('td',{'class' : 'r'})
#find( , {})사용해 div 태그 중에서 id가 'svd~'인 태그를 찾음, find_all( , {}) 사용해 td 클래스 = r 인 것 찾아줌
market_value = float(mv_cells[8].string.replace(',', '')) #8번째 인덱스에서, 699,189만 뽑아내려면 string 사용함
#에러 뜨기 때문에 콤마 삭제해줘야함, 파이썬 replace 함수 사용
market_value

699189.0

#Current Asset(유동자산)#

In [ ]:
ca_cells = fs_soup.find('div', {'id' : 'divDaechaY'}).find_all('td', {'class' : 'r cle'})
current_asset = float(ca_cells[1].string.replace(',', ''))
current_asset

52079.0

#Total Debt(총부채)#

In [ ]:

debt_cells = fs_soup.find('div', {'id' : 'divDaechaY'}).find_all('tr', {'class' : 'rwf rowBold'})

#html 크롤링 방식으로 debt 찾기

total_debt =float(debt_cells[1].find('td', {'class' : 'r cle'}).string.replace(',', '')) #부채는 1번째 인덱스(덩어리째로 1번째)
total_debt

56369.0

In [ ]:
#파이썬 방식으로 debt 찾기
debt_list = []
debt_cells[1]

for debt in debt_cells[1] :
  if debt != '\n' : #띄어쓰기 된 부분 없는 리스트 생성
    debt_list.append(debt.string)
debt_list

total_debt = float(debt_list[-1].string.replace(',','')) #-1은 마지막 인덱스
total_debt

56369.0

#Net Income(세후이익)#

In [ ]:
income_cells = fs_soup.find('div', {'id' : 'divSonikY'}).find_all('tr',{'class' :'rwf rowBold'})
net_income = float(income_cells[-1].find_all('td', {'class' : 'r'})[-3].string.replace(',','')) #find : 하나만 찾아줌, find_all : 모두 찾아줌
net_income

NameError: ignored

#Get All Stock Code#

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

csv_url = '/content/drive/MyDrive/QUANT/company.csv'

company = pd.read_csv(csv_url, encoding='utf-8')

print(company.shape)
company.head()

Mounted at /content/drive
(2022, 11)


,분류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,구글코드
0,코스피,두산우,000155,회사 본부 및 경영 컨설팅 서비스업,"청주,소주,인쇄회로기판(인쇄회로용동박적층판),관련전자부품,기계 제조,판매/출판사업,...",1973-06-29,12월,"박정원, 이재경",http://www.doosancorp.co.kr,서울특별시,KRX:000155
1,코스피,한화3우B,00088K,기타 화학제품 제조업,"화약,화공품,자동선반류,공작기계 제조,전자교환기,건설,정보통신,종합무역/임직원 및 ...",1976-06-24,12월,"최양수, 이태종, 김연철, 이민석",http://www.hanwhacorp.co.kr,서울특별시,KRX:00088K
2,코스피,S-Oil우,010955,석유 정제품 제조업,"석유제품,가스,윤활기유,윤활유,그리스,석유화학제품 제조,도매",1987-05-27,12월,Othman Al-Ghamdi,http://www.s-oil.com,서울특별시,KRX:010955
3,코스피,LG화학우,051915,기초 화학물질 제조업,"유화/기능/합성수지,재생섬유소,산업재,리튬이온전지,평광판,PVC 제조,도매",2001-04-25,12월,박진수,http://www.lgchem.com,서울특별시,KRX:051915
4,코스피,한국금융지주우,071055,기타 금융업,금융지주회사,2003-07-21,12월,김남구,http://www.koreaholdings.com,서울특별시,KRX:071055


In [ ]:
company[company['회사명']=='카카오']

,분류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,구글코드
16,코스피,카카오,035720,"자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업",인터넷 서비스(인터넷 광고),2017-07-10,12월,임지훈,http://www.kakaocorp.com,제주특별자치도,KRX:035720


In [ ]:
code_list = company['종목코드']
code_list

0       000155
1       00088K
2       010955
3       051915
4       071055
         ...  
2017    013030
2018    019550
2019    019590
2020    019570
2021    006920
Name: 종목코드, Length: 2022, dtype: object

In [ ]:
sample_code_list = code_list[1000:1050]
sample_code_list

1000    207930
1001    214270
1002    189690
1003    213420
1004    206660
1005    060480
1006    206640
1007    208640
1008    193250
1009    204630
1010    200470
1011    200670
1012    208710
1013    189860
1014    067390
1015    080580
1016    208350
1017    160550
1018    124500
1019    142280
1020    187220
1021    208370
1022    207720
1023    208140
1024    149980
1025    084650
1026    200710
1027    200780
1028    207760
1029    196170
1030    206400
1031    178920
1032    140520
1033    173940
1034    205470
1035    200230
1036    194510
1037    204840
1038    196490
1039    204620
1040    205500
1041    205100
1042    191420
1043    192440
1044    182690
1045    143540
1046    203650
1047    041920
1048    194480
1049    203690
Name: 종목코드, dtype: object

In [ ]:
company['종목코드'] == '035720'#불리안형으로 나옴
company[company['종목코드'] == '035720'] #True 값만 나옴
company[company['종목코드'] == '035720'] #array 나옴
company[company['종목코드'] == '035720']['회사명'].values[0] #회사명만 뽑아옴

'카카오'

In [ ]:
sample_dict = {}

sample_dict['카카오'] = [1,2,3,4]

sample_dict

{'카카오': [1, 2, 3, 4]}

In [ ]:
def crawler(li) :
  #url 세팅
  result = {}
  SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
  FS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'

  #for 반복문 시작
  for code in tqdm(li) :
    try :
      #for 반복문 안에서 돌리기 위해 위에서 다시 가져옴
      fs = urllib.request.urlopen(FS_URL % code).read() 
      fs_soup = BeautifulSoup(fs, 'html.parser') 

      snap = urllib.request.urlopen(SNAP_URL % code).read()
      snap_soup = BeautifulSoup(snap, 'html.parser')

      #Get company name
      name = company[company['종목코드'] == code]['회사명'].values[0]

      #Get Market Value
      mv_cells = snap_soup.find('div', {'id' : 'svdMainGrid1'}).find_all('td',{'class' : 'r'})
      market_value = float(mv_cells[8].string.replace(',', ''))

      #Get Current Asset
      ca_cells = fs_soup.find('div', {'id' : 'divDaechaY'}).find_all('td', {'class' : 'r cle'})
      current_asset = float(ca_cells[1].string.replace(',', ''))

      #Get total debt
      debt_cells = fs_soup.find('div', {'id' : 'divDaechaY'}).find_all('tr', {'class' : 'rwf rowBold'})
      total_debt =float(debt_cells[1].find('td', {'class' : 'r cle'}).string.replace(',', '')) 

      #Get net income
      income_cells = fs_soup.find('div', {'id' : 'divSonikY'}).find_all('tr',{'class' :'rwf rowBold'})
      net_income = float(income_cells[-1].find_all('td', {'class' : 'r'})[-3].string.replace(',','')) 

      #insert into result dict
      result[name] = [code, market_value, current_asset, total_debt, net_income]

    except (ValueError, AttributeError, IndexError, TypeError,) :
      pass

  # convert dict into DataFrame
  result_df = pd.DataFrame(result)

  #transposing DataFrame
  result_df = result_df.transpose()

  #Set Column Names
  result_df.columns = ['Code', 'Market_Value', 'Current_Asset', 'Total_Debt', 'Net_Income']

  return result_df


In [ ]:
result_df = crawler(sample_code_list)
result_df

100%|██████████| 50/50 [02:50<00:00,  3.41s/it]


,Code,Market_Value,Current_Asset,Total_Debt,Net_Income
퓨쳐스트림네트웍스,214270,1619,1144,1234,-4
포시에스,189690,761,194,18,25
덕산네오룩스,213420,13518,1017,283,123
국일신동,060480,599,192,52,4
바디텍메드,206640,5273,1250,355,123
썸에이지,208640,4391,347,31,140
와이제이엠게임즈,193250,1427,285,174,-13
화이브라더스코리아,204630,997,628,1040,-92
하이셈,200470,1260,198,592,-2
휴메딕스,200670,3595,894,452,40


In [ ]:
# 데이터 카피 해줘야함

tmp_df = result_df.copy()
#tmp_df

#Searching Stocks#

In [ ]:
tmp_df['NCAV'] = tmp_df['Current_Asset'] - tmp_df['Total_Debt'] #NCAV열 추가
#tmp_df

In [ ]:
tmp_df[(tmp_df['NCAV'] > tmp_df['Market_Value']*0.3) & (tmp_df['Net_Income'] > 0)]

,Code,Market_Value,Current_Asset,Total_Debt,Net_Income,NCAV
슈피겐코리아,192440,3506,2244,630,168,1614
메디아나,041920,1211,558,110,25,448
